# Imports

In [1]:
from flipside import Flipside
from config import flipside_api_key, flipside_nft_holder_address

In [2]:
import pandas as pd
import os
from datetime import datetime as dt
from datetime import timedelta


In [3]:
from app.data.local_storage import (
    read_json,
    read_csv
)


In [4]:
sdk = Flipside(flipside_api_key)

# Data Checklist

```markdown
* [X] Curve Locks
* [X] Curve Votes

* [X] Gauge to LP map
* [] Curve Liquidity

* [X] Convex Snapshot
* [X] Votium Bounties

* [X] StakeDAO Snapshot
```

# Define Save Function

In [5]:
data_path = "/app/data/source"

def get_cwd():
    cwd_temp = os.getcwd()
    temp_split = cwd_temp.split('/')
    cwd = ""
    for x in temp_split:
        if x == 'experiments':
            break
        elif x == '':
            pass
        else:
            cwd += "/"+x       
    return cwd

def print_metrics(query):
    started_at = query.run_stats.started_at
    ended_at = query.run_stats.ended_at
    elapsed_seconds = query.run_stats.elapsed_seconds
    record_count = query.run_stats.record_count
    print(f"This query took ${elapsed_seconds} seconds to run and returned {record_count} records from the database.")


def query_and_save(_query, _filename, _df_base = [], _page_size = 5000):
    try:
        # Initial Query
        if len(_df_base) > 0:
            print("based")
            df_output = _df_base
        else:
            df_output = []

        print(f"___\n{_filename}")
        print(f"querying page: 1")
        query_result_set = sdk.query(_query, page_size = _page_size)
        if len(df_output) == 0:
            df_output = pd.json_normalize(query_result_set.records)
        else:
            # Concat Dataframes
            df_local = pd.json_normalize(query_result_set.records)
            df_output = pd.concat([df_output, df_local], ignore_index=True)

        # Metrics
        print_metrics(query_result_set)
        i = 2
            
        # Handle Pagination
        if len(query_result_set.records) >= _page_size:
            keep_going = True
            while keep_going:
                print(f"querying page: {i}")
                extended_result_set = sdk.get_query_results(
                    query_result_set.query_id,
                    page_number=i,
                    page_size=_page_size
                )
                # Metrics
                print_metrics(query_result_set)
                
                # Concat Dataframes
                df_local = pd.json_normalize(extended_result_set.records)
                df_output = pd.concat([df_output, df_local], ignore_index=True)
                
                # Check if continue
                print(len(extended_result_set.records) < _page_size)
                print(len(extended_result_set.records))
                if len(extended_result_set.records) < _page_size:
                    keep_going = False
                i += 1
        # Save
        cwd = get_cwd()
        full_filename = cwd+ data_path + '/' + _filename+'.csv'
        df_output.to_csv(full_filename) 
    except Exception as e:
        print(e)
        if len(_df_base) > 0:
            df_output = _df_base
        else:
            df_output = []

    return df_output

In [6]:
def get_df_and_target(filename, target = 'block_timestamp'):
    # gets dataframe and max value of target intended for time values
    resp_dict = read_csv(filename, 'source')
    df = pd.json_normalize(resp_dict)
    print(df.keys())

    # Get Max Value of target
    temp_df = df.sort_values(target).tail(1)
    search_result = temp_df.iloc[0][target] 
    # # If date only drown out time diffs by removing last
    # if target == 'date_day':
    #     # remove most current
    #     df = df[df[target] < search_result]
    #     # find new max value of target
    #     temp_df = df.sort_values(target).tail(1)
    #     search_result = temp_df.iloc[0][target] 

    try:
        if 'T' in search_result:
            split = search_result.split("T")
            search_result = split[0]+" "+split[1][:-1]
    except:
        pass
    print(search_result)
    return df, search_result


# SQL

## Curve Locker

In [7]:
filename = 'curve_locker'

In [8]:
df, block_timestamp = get_df_and_target(filename)

Index(['', 'block_number', 'block_timestamp', 'tx_hash', 'event_index',
       'contract_address', 'contract_name', 'event_name',
       'origin_function_signature', 'origin_from_address', 'origin_to_address',
       'topics', 'data', 'event_removed', 'tx_status', 'week_number',
       'week_day', '__row_index', 'decoded_log.prevSupply',
       'decoded_log.supply', 'full_decoded_log.address',
       'full_decoded_log.data', 'full_decoded_log.decoded',
       'full_decoded_log.name', 'decoded_log.locktime', 'decoded_log.provider',
       'decoded_log.ts', 'decoded_log.type', 'decoded_log.value',
       'decoded_log.admin'],
      dtype='object')
2023-09-28 03:57:11.000


In [9]:
curve_locker_address = '0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2'

curve_locker_query = f"""SELECT 
  *,
  WEEK(BLOCK_TIMESTAMP) as WEEK_NUMBER,
  DAYOFWEEK(BLOCK_TIMESTAMP) as WEEK_DAY

FROM ethereum.core.ez_decoded_event_logs
WHERE CONTRACT_ADDRESS = lower('{curve_locker_address}')
AND BLOCK_TIMESTAMP >'{block_timestamp}'

"""


In [10]:
df_curve_locker = query_and_save(curve_locker_query, filename, df)

based
___
curve_locker
querying page: 1
This query took $11 seconds to run and returned 240 records from the database.


## Curve Gauge Votes

In [11]:
filename = 'curve_gauge_votes'

In [12]:
df, block_timstamp = get_df_and_target(filename)

Index(['', 'symbol', 'name', 'week_number', 'week_day', 'tx_hash',
       'block_timestamp', '__row_index', 'decoded_log.gauge_addr',
       'decoded_log.time', 'decoded_log.user', 'decoded_log.weight'],
      dtype='object')
2023-09-28 11:57:35.000


In [13]:
curve_voter_address = '0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB'

curve_gauge_vote_query = f"""SELECT 
    SYMBOL, 
    NAME, 
    WEEK(BLOCK_TIMESTAMP) as WEEK_NUMBER,
    DAYOFWEEK(BLOCK_TIMESTAMP) as WEEK_DAY,
    DECODED_LOG,
    TX_HASH,
    BLOCK_TIMESTAMP

FROM ethereum.core.ez_decoded_event_logs LOGS
 LEFT JOIN ethereum.core.dim_contracts CONTRACT
   ON CONTRACT.address = lower(LOGS.DECODED_LOG:gauge_addr::string)
WHERE CONTRACT_ADDRESS = lower('{curve_voter_address}')
AND EVENT_NAME = 'VoteForGauge'
AND BLOCK_TIMESTAMP >'{block_timestamp}'
ORDER BY BLOCK_TIMESTAMP ASC
"""

In [14]:
df_curve_guage_votes = query_and_save(curve_gauge_vote_query, filename, df, 5000)

based
___
curve_gauge_votes
querying page: 1
This query took $11 seconds to run and returned 291 records from the database.


## Snapshot Votes

In [15]:
filename = 'convex_snapshot_votes'
df, vote_timestamp = get_df_and_target(filename, 'vote_timestamp')

Index(['', 'proposal_id', 'proposal_start_time', 'proposal_end_time',
       'proposal_title', 'proposal_author', 'vote_option', 'voting_power',
       'vote_timestamp', 'quorum', 'choices', 'voting_period', 'network',
       'space_id', 'voter', 'address_name', 'label_type', 'label_subtype',
       'label', '__row_index'],
      dtype='object')
2023-09-19 00:00:44.000


In [16]:
convex_snapshot_votes_query = f"""
SELECT 
    PROPOSAL_ID, 
    PROPOSAL_START_TIME, 
    PROPOSAL_END_TIME, 
    PROPOSAL_TITLE, 
    PROPOSAL_AUTHOR,
    VOTE_OPTION,
    VOTING_POWER,
    VOTE_TIMESTAMP,
    QUORUM,
    CHOICES,
    VOTING_PERIOD,
    NETWORK,
    SPACE_ID,
    VOTER,
    ADDRESS_NAME,
    LABEL_TYPE, 
    LABEL_SUBTYPE,
    LABEL
FROM external.snapshot.ez_snapshot as SNAPSHOT
 LEFT JOIN ethereum.core.dim_labels LABELS
   ON SNAPSHOT.VOTER = LABELS.ADDRESS
WHERE SPACE_ID = 'cvx.eth' 
AND PROPOSAL_TITLE LIKE 'Gauge Weight%'
AND VOTE_TIMESTAMP > '{vote_timestamp}'
"""

In [17]:
df_snapshot_votes = query_and_save(convex_snapshot_votes_query, filename, df)

based
___
convex_snapshot_votes
querying page: 1
This query took $9 seconds to run and returned 204 records from the database.


In [18]:
filename = 'stakedao_snapshot'
df, vote_timestamp = get_df_and_target(filename, 'vote_timestamp')

Index(['', 'proposal_id', 'proposal_start_time', 'proposal_end_time',
       'proposal_title', 'proposal_author', 'vote_option', 'voting_power',
       'vote_timestamp', 'quorum', 'choices', 'voting_period', 'network',
       'space_id', 'voter', 'address_name', 'label_type', 'label_subtype',
       'label', '__row_index'],
      dtype='object')
2023-09-18 11:42:24.000


In [19]:
stakedao_snapshot_votes_query = f"""
SELECT 
    PROPOSAL_ID, 
    PROPOSAL_START_TIME, 
    PROPOSAL_END_TIME, 
    PROPOSAL_TITLE, 
    PROPOSAL_AUTHOR,
    VOTE_OPTION,
    VOTING_POWER,
    VOTE_TIMESTAMP,
    QUORUM,
    CHOICES,
    VOTING_PERIOD,
    NETWORK,
    SPACE_ID,
    VOTER,
    ADDRESS_NAME,
    LABEL_TYPE, 
    LABEL_SUBTYPE,
    LABEL
FROM external.snapshot.ez_snapshot as SNAPSHOT
 LEFT JOIN ethereum.core.dim_labels LABELS
   ON SNAPSHOT.VOTER = LABELS.ADDRESS
WHERE SPACE_ID = 'sdcrv.eth' 
AND (PROPOSAL_TITLE LIKE 'Gauge vote - CRV%'
OR
PROPOSAL_TITLE LIKE 'Gauge vote CRV%'
)
AND VOTE_TIMESTAMP > '{vote_timestamp}'

"""

In [20]:
df_snapshot_votes = query_and_save(stakedao_snapshot_votes_query, filename, df,  1500)

based
___
stakedao_snapshot
querying page: 1
This query took $10 seconds to run and returned 9 records from the database.


In [21]:
df_snapshot_votes

,,proposal_id,proposal_start_time,proposal_end_time,proposal_title,proposal_author,vote_option,voting_power,vote_timestamp,quorum,choices,voting_period,network,space_id,voter,address_name,label_type,label_subtype,label,__row_index
0,0,0x9988c5a62fb5a44d351fb1e6fa76eea9f944f4680a0a...,2023-03-16T00:00:00.000Z,2023-03-20T14:00:00.000Z,Gauge vote - CRV 23/3/2023 - 5/4/2023,0x0657c6bee67bb96fae96733d083daade0cb5a179,"['{""119"":15,""159"":5,""262"":10,""327"":10,""71"":60}']",1770001,2023-03-20T07:45:11.000Z,,"['[""AAVE+palStkAAVE (0x4853…6B15)"",""aDAI+aSUSD...",,Ethereum Mainnet,sdcrv.eth,0xf930ebbd05ef8b25b1797b9b2109ddc9b0d43063,gnosis safe: general contract,dapp,general_contract,gnosis safe,0
1,1,0xb99f49298447714103746ccd4370f57a300ed4f54925...,2023-08-03T00:00:00.000Z,2023-08-07T13:00:00.000Z,Gauge vote - CRV 10/8/2023 - 23/8/2023,0x0657c6bee67bb96fae96733d083daade0cb5a179,"['{""104"":5,""13"":5,""154"":5,""180"":15,""230"":5,""37...",2843601,2023-08-06T09:21:12.000Z,,"['[""aDAI+aSUSD (0xEB16…a733) - 0x462253b8f74b7...",,Ethereum Mainnet,sdcrv.eth,0xf930ebbd05ef8b25b1797b9b2109ddc9b0d43063,gnosis safe: general contract,dapp,general_contract,gnosis safe,1
2,2,0xbff6ff0e253ee471e78cf46c207e99ffcf7bb7e7d846...,2023-03-02T00:00:00.000Z,2023-03-06T14:00:00.000Z,Gauge vote - CRV 9/3/2023 - 22/3/2023,0x0657c6bee67bb96fae96733d083daade0cb5a179,"['{""115"":15,""12"":5,""154"":5,""255"":10,""317"":10,""...",1802378,2023-03-06T09:43:41.000Z,,"['[""AAVE+palStkAAVE (0x4853…6B15)"",""aDAI+aSUSD...",,Ethereum Mainnet,sdcrv.eth,0xf930ebbd05ef8b25b1797b9b2109ddc9b0d43063,gnosis safe: general contract,dapp,general_contract,gnosis safe,2
3,3,0x0c257e30b2d519533a0b73b907f631d879fcf1a1f9e4...,2023-07-20T00:00:00.000Z,2023-07-24T13:00:00.000Z,Gauge vote - CRV 27/7/2023 - 9/8/2023,0x0657c6bee67bb96fae96733d083daade0cb5a179,"['{""102"":5,""13"":5,""147"":5,""172"":10,""220"":5,""37...",2208661,2023-07-22T09:19:02.000Z,,"['[""aDAI+aSUSD (0xEB16…a733) - 0x462253b8f74b7...",,Ethereum Mainnet,sdcrv.eth,0xf930ebbd05ef8b25b1797b9b2109ddc9b0d43063,gnosis safe: general contract,dapp,general_contract,gnosis safe,3
4,4,0x84f3a96fb35e7febed9c6021e80c09ebb4553838c9d7...,2023-06-08T00:00:00.000Z,2023-06-12T13:00:00.000Z,Gauge vote - CRV 15/6/2023 - 29/6/2023,0x0657c6bee67bb96fae96733d083daade0cb5a179,"['{""113"":15,""175"":5,""176"":55,""177"":5,""178"":5,""...",1850066,2023-06-11T11:56:35.000Z,,"['[""cDAI+cUSDC (0xA2B4…7A56)"",""cDAI+cUSDC+USDT...",,Ethereum Mainnet,sdcrv.eth,0xf930ebbd05ef8b25b1797b9b2109ddc9b0d43063,gnosis safe: general contract,dapp,general_contract,gnosis safe,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,NaN,0x079118fb410afa49270afdd25fa69835eb884c9f9948...,2023-09-28T01:00:00.000Z,2023-10-02T13:00:00.000Z,Gauge vote CRV - 5/10/2023 - 18/10/2023,0x0657c6bee67bb96fae96733d083daade0cb5a179,"[{""30"":1,""89"":4,""99"":2}]",3615657,2023-09-28T08:03:20.000Z,None,"[[""cDAI+cUSDC (0xA2B4…7A56) - 0x7ca5b0a2910b33...",None,ethereum,sdcrv.eth,0xf872703f1c8f93fa186869bac83bac5a0c87c3c8,None,None,None,None,4
220,NaN,0x079118fb410afa49270afdd25fa69835eb884c9f9948...,2023-09-28T01:00:00.000Z,2023-10-02T13:00:00.000Z,Gauge vote CRV - 5/10/2023 - 18/10/2023,0x0657c6bee67bb96fae96733d083daade0cb5a179,"[{""169"":1,""198"":1,""221"":1,""254"":1,""255"":1,""256...",1934966,2023-09-29T00:04:26.000Z,None,"[[""cDAI+cUSDC (0xA2B4…7A56) - 0x7ca5b0a2910b33...",None,ethereum,sdcrv.eth,0x7a16ff8270133f063aab6c9977183d9e72835428,None,None,None,None,5
221,NaN,0x079118fb410afa49270afdd25fa69835eb884c9f9948...,2023-09-28T01:00:00.000Z,2023-10-02T13:00:00.000Z,Gauge vote CRV - 5/10/2023 - 18/10/2023,0x0657c6bee67bb96fae96733d083daade0cb5a179,"[{""57"":1}]",734,2023-09-28T08:31:21.000Z,None,"[[""cDAI+cUSDC (0xA2B4…7A56) - 0x7ca5b0a2910b33...",None,ethereum,sdcrv.eth,0x8a97ff08958a4637c85939d13b29b5dca6dba977,None,None,None,None,6
222,NaN,0x079118fb410afa49270afdd25fa69835eb884c9f9948...,2023-09-28T01:00:00.000Z,2023-10-02T13:00

# Votium Bounties

In [22]:
filename = 'votium_bounty_for_round'
df, block_timestamp = get_df_and_target(filename)

Index(['', 'event_name', 'tx_hash', 'choice_index', 'amount', 'proposal_id',
       'bounty_token_address', 'origin_from_address', 'block_timestamp',
       'price', 'bounty_value', 'token_name', 'token_symbol', '__row_index'],
      dtype='object')
2023-09-04 17:30:59.000


In [23]:
votium_bounty_query = f"""
SELECT 
  EVENT_NAME,
  TX_HASH,
  DECODED_LOG:_choiceIndex::int as choice_index,
  DECODED_LOG:_amount::int / pow(10,18) as amount,
  DECODED_LOG:_proposal::string as proposal_id,
  DECODED_LOG:_token::string as bounty_token_address,
  ORIGIN_FROM_ADDRESS,
  BLOCK_TIMESTAMP as block_timestamp,
  PRICE as price,
  price * amount as bounty_value,
  contracts.NAME as token_name,
  contracts.SYMBOL as token_symbol
FROM ethereum.core.ez_decoded_event_logs as logs

LEFT JOIN ethereum.core.fact_hourly_token_prices as prices
ON logs.DECODED_LOG:_token = prices.token_address
AND time_slice(logs.BLOCK_TIMESTAMP::timestamp_ntz, 1, 'HOUR') = prices.HOUR

LEFT JOIN ethereum.core.dim_contracts as contracts
ON logs.DECODED_LOG:_token = contracts.ADDRESS

WHERE CONTRACT_ADDRESS= lower('0x19bbc3463dd8d07f55438014b021fb457ebd4595')

AND EVENT_NAME = 'Bribed'
AND BLOCK_TIMESTAMP > '{block_timestamp}'


ORDER BY block_timestamp
"""

In [24]:
df_votium_bounty = query_and_save(votium_bounty_query, filename, df)

based
___
votium_bounty_for_round
querying page: 1
4 validation errors for GetQueryRunResultsRpcResponse
result -> rows
  none is not an allowed value (type=type_error.none.not_allowed)
result -> page
  none is not an allowed value (type=type_error.none.not_allowed)
result -> sql
  none is not an allowed value (type=type_error.none.not_allowed)
result -> format
  none is not an allowed value (type=type_error.none.not_allowed)


# Reference: Gauge to LP Map

In [25]:
filename = 'gauge_to_lp_map'
df, block_timestamp = get_df_and_target(filename, 'deployed_timestamp')


Index(['', 'type_id', 'type_name', 'name', 'symbol', 'gauge_addr', 'weight',
       'type_weight', 'type_total_weight', 'type_weight_time', 'tx_hash',
       'vote_timestamp', 'pool_addr', 'token_addr', 'source',
       'deployed_timestamp', 'gauge_name', 'gauge_symbol', 'pool_name',
       'pool_symbol', 'token_name', 'token_symbol', 'chain_id', '__row_index'],
      dtype='object')
2023-09-29 19:32:47.000


In [32]:
curve_voter_address = '0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB'

gauge_to_lp_map_query = f"""
-- 0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB
with gauge_types as (
  SELECT 
    DECODED_LOG:name::string as name,
    DECODED_LOG:type_id::int as type_id,
    BLOCK_TIMESTAMP as block_timestamp,
    TX_HASH as tx_hash
  
  FROM ethereum.core.ez_decoded_event_logs LOGS
   LEFT JOIN ethereum.core.dim_contracts CONTRACT
     ON CONTRACT.address = lower(LOGS.DECODED_LOG:gauge_addr::string)
  WHERE CONTRACT_ADDRESS = lower('{curve_voter_address}')
  AND EVENT_NAME = 'AddType'
),

new_type_weight as (
  SELECT 
    ifnull(DECODED_LOG:time::int, 0) as time,
    DECODED_LOG:total_weight::string as total_weight,
    ifnull(DECODED_LOG:type_id::int, 0) as type_id,
    DECODED_LOG:weight::string as weight,
    TX_HASH,
    BLOCK_TIMESTAMP
  
  FROM ethereum.core.ez_decoded_event_logs
  WHERE CONTRACT_ADDRESS = lower('{curve_voter_address}')
  AND EVENT_NAME = 'NewTypeWeight'
  ORDER BY BLOCK_TIMESTAMP ASC

),

new_gauges as (

  SELECT 
      SYMBOL, 
      NAME, 
      DECODED_LOG:addr::string as gauge_addr,
      DECODED_LOG:gauge_type::int as gauge_type,
      DECODED_LOG:weight::int as weight,
      TX_HASH,
      BLOCK_TIMESTAMP
  
  FROM ethereum.core.ez_decoded_event_logs LOGS
   LEFT JOIN ethereum.core.dim_contracts CONTRACT
     ON CONTRACT.address = lower(LOGS.DECODED_LOG:addr::string)
  WHERE CONTRACT_ADDRESS = lower('{curve_voter_address}')
  AND EVENT_NAME = 'NewGauge'
  ORDER BY BLOCK_TIMESTAMP ASC
  
),

gauge_meta as (
  SELECT 
    new_gauges.gauge_type as type_id,
    gauge_types.name as type_name,
    new_gauges.name,
    new_gauges.symbol,
    new_gauges.gauge_addr,
    new_gauges.weight,
    new_type_weight.weight as type_weight,
    new_type_weight.total_weight as type_total_weight,
    new_type_weight.time as type_weight_time,
    new_gauges.tx_hash,
    new_gauges.block_timestamp as vote_timestamp
  FROM new_gauges 
  LEFT JOIN gauge_types
  ON new_gauges.gauge_type = gauge_types.type_id
  LEFT JOIN new_type_weight
  ON new_gauges.gauge_type = new_type_weight.type_id
  ORDER BY new_gauges.block_timestamp DESC
),


-- BREAK between meta and deployers

v2_deployer as (
  SELECT
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    DECODED_LOG:token::string as token_addr,
    'v2' as source
  
  
  FROM ethereum.core.ez_decoded_event_logs
  WHERE CONTRACT_ADDRESS = lower('0xF18056Bbd320E96A48e3Fbf8bC061322531aac99') -- v2 deployer
  AND EVENT_NAME = 'LiquidityGaugeDeployed'
  AND BLOCK_TIMESTAMP > '{block_timestamp}'

),

factory_deployer as (
  SELECT 
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    '' as token_addr,
    'factory' as source

  FROM ethereum.core.ez_decoded_event_logs as logs

  WHERE logs.CONTRACT_ADDRESS = lower('0xB9fC157394Af804a3578134A6585C0dc9cc990d4')  -- factory
  AND logs.EVENT_NAME = 'LiquidityGaugeDeployed'
  AND BLOCK_TIMESTAMP > '{block_timestamp}'

),


stable_deployer as (
  SELECT 
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    '' as token_addr,
    'stable_factory' as source

  FROM ethereum.core.ez_decoded_event_logs as logs

  WHERE logs.CONTRACT_ADDRESS = lower('0x4F8846Ae9380B90d2E71D5e3D042dff3E7ebb40d')  -- factory
  AND logs.EVENT_NAME = 'LiquidityGaugeDeployed'
  AND BLOCK_TIMESTAMP > '{block_timestamp}'

),


multichain_deployer as (
  SELECT 
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:_gauge::string as gauge_addr,
    DECODED_LOG:_chain_id::string as chain_id,
    DECODED_LOG:_deployer::string as deployer,
    DECODED_LOG:_implementation::string as implementation,
    DECODED_LOG:_salt::string as salt,
    '' as pool_addr,
    '' as token_addr,
    'multichain_factory' as source

  FROM ethereum.core.ez_decoded_event_logs as logs

  WHERE logs.CONTRACT_ADDRESS = lower('0xabc000d88f23bb45525e447528dbf656a9d55bf5')  -- factory
  AND logs.EVENT_NAME = 'DeployedGauge'
  AND BLOCK_TIMESTAMP > '{block_timestamp}'

),


tricrypto_deployer as (
  SELECT 
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    '' as token_addr,
    'tricrypto_factory' as source

  FROM ethereum.core.ez_decoded_event_logs as logs

  WHERE logs.CONTRACT_ADDRESS = lower('0x0c0e5f2ff0ff18a3be9b835635039256dc4b4963')  -- tricrypto_factory
  AND logs.EVENT_NAME = 'LiquidityGaugeDeployed'
  AND BLOCK_TIMESTAMP > '{block_timestamp}'

),


combo as (
  SELECT gauge_addr, pool_addr, token_addr, source, block_timestamp, '' as chain_id FROM v2_deployer
  UNION
  SELECT gauge_addr, pool_addr, token_addr, source, block_timestamp,'' as chain_id   FROM factory_deployer
  UNION
  SELECT gauge_addr, pool_addr, token_addr, source, block_timestamp,'' as chain_id FROM stable_deployer
  UNION
  SELECT gauge_addr, pool_addr, token_addr, source, block_timestamp,'' as chain_id FROM tricrypto_deployer
  UNION
  SELECT gauge_addr, pool_addr, token_addr, source, block_timestamp, chain_id  FROM multichain_deployer

),

deployer_meta as (
  SELECT 
    combo.gauge_addr as gauge_addr, 
    combo.pool_addr as pool_addr, 
    combo.token_addr as token_addr, 
    combo.source as source, 
    combo.chain_id as chain_id,
    combo.block_timestamp as block_timestamp,
    contracts.NAME as gauge_name,
    contracts.SYMBOL as gauge_symbol,
    pool_contracts.NAME as pool_name,
    pool_contracts.SYMBOL as pool_symbol,
    token_contracts.NAME as token_name,
    token_contracts.SYMBOL as token_symbol
  FROM combo
  LEFT JOIN ethereum.core.dim_contracts as contracts
    ON combo.gauge_addr = contracts.ADDRESS
  LEFT JOIN ethereum.core.dim_contracts as pool_contracts
    ON combo.pool_addr = pool_contracts.ADDRESS
  LEFT JOIN ethereum.core.dim_contracts as token_contracts
    ON combo.token_addr = token_contracts.ADDRESS
)


SELECT 
    gauge_meta.type_id,
    gauge_meta.type_name,
    IFNULL(gauge_meta.name, deployer_meta.gauge_name) as name,
    IFNULL(gauge_meta.symbol, deployer_meta.gauge_symbol) as symbol,
    IFNULL(gauge_meta.gauge_addr, deployer_meta.gauge_addr) as gauge_addr,
    gauge_meta.weight,
    gauge_meta.type_weight,
    gauge_meta.type_total_weight,
    gauge_meta.type_weight_time,
    gauge_meta.tx_hash,
    gauge_meta.vote_timestamp,

    deployer_meta.pool_addr, 
    deployer_meta.token_addr, 
    deployer_meta.source, 
    deployer_meta.block_timestamp as deployed_timestamp,
    deployer_meta.gauge_name,
    deployer_meta.gauge_symbol,
    deployer_meta.pool_name,
    deployer_meta.pool_symbol,
    deployer_meta.token_name,
    deployer_meta.token_symbol,
    deployer_meta.chain_id
FROM gauge_meta
FULL JOIN deployer_meta
ON gauge_meta.gauge_addr = deployer_meta.gauge_addr
ORDER BY deployed_timestamp DESC
"""

In [33]:
df_curve_gauge_map = query_and_save(gauge_to_lp_map_query, filename, df)

based
___
liquidity_general
querying page: 1
This query took $16 seconds to run and returned 349 records from the database.


In [34]:
df_curve_gauge_map

,,date_day,pool_name,current_bal,current_bal_usd,tradeable_assets,pool_address,__row_index,type_id,type_name,...,pool_addr,token_addr,source,deployed_timestamp,gauge_name,gauge_symbol,pool_symbol,token_name,token_symbol,chain_id
0,516,2023-09-19T00:00:00.000Z,Curve.fi Factory Plain Pool: msETH/WETH,1937.734040511,1255703.2770894,"msETH, WETH, ETH",0xa4c567c662349bec3d0fb94c4e7f85ba95e208e4,516,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,517,2023-09-19T00:00:00.000Z,Curve.fi Factory USD Metapool: GHOFRAXBP,2398.044261744,2342.247240794,"crvFRAX, GHO, ETH, USDC",0xbc90fec043e6df6a084e18df9435ee037c940b2d,517,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,518,2023-09-19T00:00:00.000Z,Curve.fi Factory USD Metapool: RC_NEAR_1.1_DAI...,51.293717304,,"DAI, RR_NEAR_1.1_DAI_2021_6_30, ETH, RC_NEAR_1...",0x620e3c54d8c6efca7476d657c57da5eb144d3f81,518,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,519,2023-09-19T00:00:00.000Z,Curve.fi Factory USD Metapool: TrueUSD,1295758.95885885,1317265.47040115,"3Crv, variableDebtUSDT, aUSDC, USDC, variableD...",0xecd5e75afb02efa118af914515d6521aabd189f1,519,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,520,2023-09-19T00:00:00.000Z,ibCHF-USDC,4343.109504652,2354.103175734,"USDC, ETH, ibCHF",0x6df0d77f0496ce44e72d695943950d8641fca5cf,520,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109799,NaN,NaN,TryLSD,NaN,NaN,NaN,NaN,344,NaN,None,...,0x2570f1bd5d2735314fc102eb12fc1afe9e6e7193,,tricrypto_factory,2023-10-01T10:20:59.000Z,None,None,TryLSD,None,None,
109800,NaN,NaN,None,NaN,NaN,NaN,NaN,345,NaN,None,...,0x5b3ba844b3859f56524e99ae54857b36c8ae3efe,,tricrypto_factory,2023-10-01T03:23:11.000Z,None,None,None,None,None,
109801,NaN,NaN,None,NaN,NaN,NaN,NaN,346,NaN,None,...,0x707eae1ccfee0b8fef07d3f18eafd1246762d587,,stable_factory,2023-09-30T13:46:35.000Z,None,None,None,None,None,
109802,NaN,NaN,None,NaN,NaN,NaN,NaN,347,NaN,None,...,0xb9ec78bd89d3ef17537f130cc72750fd4de85f82,,stable_factory,2023-09-30T13:42:59.000Z,None,None,None,None,None,


# Curve Liquidity

In [28]:
filename = 'liquidity_general'
df, date_day = get_df_and_target(filename, 'date_day')

Index(['', 'date_day', 'pool_name', 'current_bal', 'current_bal_usd',
       'tradeable_assets', 'pool_address', '__row_index'],
      dtype='object')
2023-09-29 00:00:00.000


In [29]:
# date_day.date() + timedelta(days=1)
# tomorow_date = dt.strptime(date_day,'%Y-%m-%d %H:%M:%S.%f') + timedelta(days=1)

# print(date_day)
# print(tomorow_date)

df = df[df['date_day'] < date_day]
df 


,,date_day,pool_name,current_bal,current_bal_usd,tradeable_assets,pool_address,__row_index
0,516,2023-09-19T00:00:00.000Z,Curve.fi Factory Plain Pool: msETH/WETH,1937.734040511,1255703.2770894,"msETH, WETH, ETH",0xa4c567c662349bec3d0fb94c4e7f85ba95e208e4,516
1,517,2023-09-19T00:00:00.000Z,Curve.fi Factory USD Metapool: GHOFRAXBP,2398.044261744,2342.247240794,"crvFRAX, GHO, ETH, USDC",0xbc90fec043e6df6a084e18df9435ee037c940b2d,517
2,518,2023-09-19T00:00:00.000Z,Curve.fi Factory USD Metapool: RC_NEAR_1.1_DAI...,51.293717304,,"DAI, RR_NEAR_1.1_DAI_2021_6_30, ETH, RC_NEAR_1...",0x620e3c54d8c6efca7476d657c57da5eb144d3f81,518
3,519,2023-09-19T00:00:00.000Z,Curve.fi Factory USD Metapool: TrueUSD,1295758.95885885,1317265.47040115,"3Crv, variableDebtUSDT, aUSDC, USDC, variableD...",0xecd5e75afb02efa118af914515d6521aabd189f1,519
4,520,2023-09-19T00:00:00.000Z,ibCHF-USDC,4343.109504652,2354.103175734,"USDC, ETH, ibCHF",0x6df0d77f0496ce44e72d695943950d8641fca5cf,520
...,...,...,...,...,...,...,...,...
109973,,2023-09-20T00:00:00.000Z,FXN-WETH,,,"ETH, FXN, WETH",0xc15f285679a1ef2d25f53d4cbd0265e1d02f2a92,5225
109974,,2023-09-20T00:00:00.000Z,Curve.fi Factory USD Metapool: tusd-3pool,,,"USDC, DAI, ETH",0xac5f019a302c4c8caac0a7f28183ac62e6e80034,5226
109975,,2023-09-20T00:00:00.000Z,Curve.fi Factory USD Metapool: RC_WETH_2000_DA...,384.348572141,118.474511952,"USDC, 3Crv, DAI, RC_WETH_2000_DAI_2021_6_30, R...",0x56680fdebdd3e31f79938fa1222bfea4706a0758,5227
109976,,2023-09-20T00:00:00.000Z,Curve.fi Factory USD Metapool: RC_xSUSHI_7_DAI...,26.190289987,,"DAI, RR_xSUSHI_7_DAI_2021_5_31, RC_xSUSHI_7_DA...",0x4eb0bb03a246b955d36316a96be314885c23a1b0,5228


In [30]:
start_date = '2023-03-01 00:00:00.000'
curve_liquidity_query = f"""
with curve_swaps as (
SELECT
*
FROM ethereum.core.ez_dex_swaps
WHERE PLATFORM = 'curve' 
),

tokens_out as (
SELECT 
DISTINCT TOKEN_OUT as token,
CONTRACT_ADDRESS as pool_address,
POOL_NAME as pool_name,
SYMBOL_OUT as symbol
FROM curve_swaps
GROUP BY TOKEN_OUT, SYMBOL_OUT, CONTRACT_ADDRESS, POOL_NAME
),

tokens_in as (
SELECT 
DISTINCT TOKEN_IN as token,
CONTRACT_ADDRESS as pool_address,
POOL_NAME as pool_name,

SYMBOL_IN  as symbol
FROM curve_swaps
-- GROUP BY TOKEN_IN
),

pools as (
SELECT
DISTINCT CONTRACT_ADDRESS as pool_address,
POOL_NAME as pool_name,
'ETH' as symbol,
'' as token
FROM curve_swaps
-- GROUP BY CONTRACT_ADDRESS

),

tokens as (
select 
*
FROM tokens_out
UNION all
select 
*
FROM tokens_in
UNION all
select 
token, pool_address, pool_name, symbol
FROM pools
),

all_tokens as (
select 
* 
from tokens
group by token, symbol, pool_address, pool_name
),


-- COMBINE TOKENS WITH DATES
dates as (
select 
date_day 
from ethereum.core.dim_dates 
where date_day between '{date_day}' and current_date()
),

dates_x_tokens as (
select 
date_day,
symbol,
token,
pool_address,
pool_name

from dates
cross join all_tokens
ORDER BY date_day DESC
),


dates_x_tokens_x_price as (
select 
dates_x_tokens.date_day,
dates_x_tokens.symbol,
dates_x_tokens.token,
pool_address,
pool_name,
AVG(prices.PRICE) as daily_price
FROM dates_x_tokens
LEFT JOIN ethereum.core.fact_hourly_token_prices as prices
ON (dates_x_tokens.token = prices.TOKEN_ADDRESS 
  OR dates_x_tokens.token = prices.SYMBOL
  )
AND dates_x_tokens.date_day = prices.hour::date
GROUP BY (
  dates_x_tokens.date_day,
  dates_x_tokens.symbol,
  dates_x_tokens.token,
  dates_x_tokens.pool_address,
  dates_x_tokens.pool_name
  )
ORDER BY date_day DESC

),


balances as (
select

dates.date_day,
dates.pool_address,
dates.pool_name, 
dates.token,
dates.symbol,
sum(deltas.BAL_DELTA) as balance,
(balance * dates.daily_price) as balance_usd 

from dates_x_tokens_x_price as dates
left join ethereum.core.ez_balance_deltas as deltas
ON deltas.BLOCK_TIMESTAMP::date <= dates.date_day
AND deltas.USER_ADDRESS = dates.pool_address
AND (deltas.CONTRACT_ADDRESS = dates.token
OR dates.symbol = deltas.SYMBOL)
GROUP BY (
  dates.date_day,
  dates.pool_address,
  dates.pool_name, 
  dates.token,
  dates.symbol,
  dates.daily_price
  )
ORDER BY date_day DESC

) 


SELECT
  date_day, 
  pool_name,
  sum(balance) as current_bal,
  sum(balance_usd) as current_bal_usd,
  LISTAGG(symbol, ', ') as tradeable_assets,
  pool_address
FROM balances
GROUP BY date_day, pool_address, pool_name
ORDER BY date_day DESC

"""

In [31]:
df_liquidity_general = query_and_save(curve_liquidity_query, filename, df)

based
___
liquidity_general
querying page: 1
This query took $69 seconds to run and returned 4224 records from the database.
